In [2]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv("rosbank_train.csv")
raw_data.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 10 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_flag     490513 non-null int64
target_sum      490513 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 37.4+ MB


In [5]:
cl_ids_test = np.random.choice(raw_data.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)

In [6]:
transactions_train = raw_data[~raw_data.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in train dataset: ", len(transactions_train))

transactions_test = raw_data[raw_data.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in test dataset: ", len(transactions_test))

Total transactions in train dataset:  388926
Total transactions in test dataset:  101587


In [7]:
transactions_train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [8]:
transactions_train.trx_category.value_counts()

POS               329520
DEPOSIT            17105
WD_ATM_ROS         15312
WD_ATM_PARTNER      7894
C2C_IN              6180
WD_ATM_OTHER        5543
C2C_OUT             4267
BACK_TRX            2120
CAT                  955
CASH_ADV              30
Name: trx_category, dtype: int64

In [9]:
transactions_train.MCC.value_counts()

5411    95783
6011    43445
5814    32655
5812    23549
5499    21543
5541    15880
5912    14850
5999    10319
6012     8003
5921     6805
5331     5997
4121     5227
4829     5182
5211     4900
5691     4067
5261     3898
5977     3380
4111     3270
5200     2827
5732     2666
5311     2540
5533     2422
5651     2325
4812     2255
5641     2162
5722     2145
8099     2116
5941     2072
5661     2036
8999     2026
        ...  
4457        1
3628        1
3236        1
3057        1
3655        1
5962        1
3029        1
3181        1
5998        1
3535        1
3548        1
3387        1
3513        1
3640        1
7277        1
7631        1
3659        1
5937        1
3634        1
3508        1
3515        1
7217        1
5561        1
3191        1
3703        1
7012        1
3625        1
8675        1
3051        1
5978        1
Name: MCC, Length: 335, dtype: int64

In [10]:
transactions_train.currency.value_counts(normalize = True).head(5)

810    0.973638
978    0.010822
840    0.004029
933    0.001216
764    0.000890
Name: currency, dtype: float64

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

In [12]:
def transaction_by_hour_count(dataset):
    dataset['hour_of_transaction'] = dataset['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%H') ) 
    return dataset.groupby(['cl_id','hour_of_transaction'])['hour_of_transaction'].size().unstack().fillna(0).reset_index()

In [13]:
hour_data = transaction_by_hour_count(transactions_train)

In [14]:
hour_data.describe()

hour_of_transaction,cl_id,00,01,02,03,04,05,06,07,08,...,14,15,16,17,18,19,20,21,22,23
count,4000.000000,4000.000000,4000.0000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0000
mean,5152.375000,85.638250,0.0540,0.073500,0.087000,0.137000,0.205500,0.271500,0.404000,0.562750,...,0.876000,0.824500,0.753000,0.763500,0.708000,0.537750,0.360500,0.256500,0.174250,0.0950
std,2983.662959,73.099742,0.3881,0.624256,0.520575,0.695235,0.863977,1.031772,1.375593,1.593486,...,1.664126,1.716956,1.553092,1.666662,1.817295,1.430764,1.100383,0.890453,0.705345,0.4561
min,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
25%,2591.250000,24.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
50%,5091.000000,70.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
75%,7787.250000,129.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000
max,10215.000000,711.000000,10.0000,28.000000,8.000000,14.000000,15.000000,19.000000,19.000000,33.000000,...,21.000000,38.000000,16.000000,27.000000,41.000000,22.000000,20.000000,15.000000,13.000000,7.0000


In [15]:
hour_data['cl_id'] = hour_data.cl_id.astype('object')

In [16]:
hour_data.describe()

hour_of_transaction,00,01,02,03,04,05,06,07,08,09,...,14,15,16,17,18,19,20,21,22,23
count,4000.000000,4000.0000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0000
mean,85.638250,0.0540,0.073500,0.087000,0.137000,0.205500,0.271500,0.404000,0.562750,0.735750,...,0.876000,0.824500,0.753000,0.763500,0.708000,0.537750,0.360500,0.256500,0.174250,0.0950
std,73.099742,0.3881,0.624256,0.520575,0.695235,0.863977,1.031772,1.375593,1.593486,1.717603,...,1.664126,1.716956,1.553092,1.666662,1.817295,1.430764,1.100383,0.890453,0.705345,0.4561
min,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
25%,24.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
50%,70.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
75%,129.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000
max,711.000000,10.0000,28.000000,8.000000,14.000000,15.000000,19.000000,19.000000,33.000000,36.000000,...,21.000000,38.000000,16.000000,27.000000,41.000000,22.000000,20.000000,15.000000,13.000000,7.0000


In [17]:
def transaction_by_day_count(dataset):
    dataset['day_of_transaction'] = dataset['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%w') ) 
    tmp_df = dataset.groupby(['cl_id','day_of_transaction'])['day_of_transaction'].size().unstack().fillna(0).reset_index()
    old_columns = [old_col for old_col in tmp_df.columns.tolist() if old_col in dataset['day_of_transaction'].unique()]
    tmp_df.rename(columns={old_col: 'day_' + old_col for old_col in old_columns}, inplace=True)
    return tmp_df

In [18]:
transactions_train['day_of_transaction'] = transactions_train['TRDATETIME'].apply( lambda x: (datetime.datetime.strptime(x, '%d%b%y:%H:%M:%S')).strftime('%w') ) 

In [19]:
transactions_train['day_of_transaction'] = transactions_train.day_of_transaction.astype('int')

In [26]:
def week_day_end(row):
    if row['day_of_transaction']>=0 and row['day_of_transaction']<=4:
        return 1
    elif row['day_of_transaction']>=5:
        return 2

In [27]:
transactions_train['budnii_vyhodnoi'] = transactions_train.apply(week_day_end, axis=1)
transactions_train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,hour_of_transaction,day_of_transaction,budnii_vyhodnoi
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0,00,6,2
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0,12,4,1
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0,00,2,1
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0,00,6,2
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0,13,2,1


In [28]:
transactions_train.budnii_vyhodnoi.value_counts()

1    267846
2    121080
Name: budnii_vyhodnoi, dtype: int64